In [4]:
"""
Technical Analysis Functions for Stock Selection System
Uses yfinance API for price data
Implements all 6 technical indicators for 6-24 month investment horizon
"""

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# ============================================================================
# CORE DATA FETCHING
# ============================================================================

def get_stock_data(ticker, period="1y"):
    """
    Fetch historical price data from yfinance
    
    Args:
        ticker (str): Stock ticker symbol
        period (str): Data period - '1y', '2y', '6mo', etc.
    
    Returns:
        DataFrame: Historical OHLCV data
    """
    try:
        stock = yf.Ticker(ticker)
        df = stock.history(period=period)
        
        if df.empty:
            print(f"No data found for {ticker}")
            return None
            
        return df
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None


# ============================================================================
# INDICATOR 1: MOVING AVERAGES
# ============================================================================

def calculate_moving_averages(df):
    """
    Calculate 20, 50, and 200-day SMAs
    
    Args:
        df (DataFrame): Price data with 'Close' column
    
    Returns:
        DataFrame: Original data with MA columns added
    """
    df = df.copy()
    
    # Simple Moving Averages
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['SMA_200'] = df['Close'].rolling(window=200).mean()
    
    # Exponential Moving Averages (for reference)
    df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    
    return df


def analyze_moving_averages(df):
    """
    Analyze moving average signals
    
    Returns:
        dict: MA analysis with signals and scores
    """
    if df is None or df.empty:
        return None
    
    latest = df.iloc[-1]
    current_price = latest['Close']
    
    # Check conditions
    above_50 = current_price > latest['SMA_50']
    above_200 = current_price > latest['SMA_200']
    golden_cross = latest['SMA_50'] > latest['SMA_200']
    
    # Distance from MAs (as percentage)
    dist_20 = ((current_price - latest['SMA_20']) / latest['SMA_20'] * 100) if pd.notna(latest['SMA_20']) else None
    dist_50 = ((current_price - latest['SMA_50']) / latest['SMA_50'] * 100) if pd.notna(latest['SMA_50']) else None
    dist_200 = ((current_price - latest['SMA_200']) / latest['SMA_200'] * 100) if pd.notna(latest['SMA_200']) else None
    
    # Score (0-3)
    score = sum([above_50, above_200, golden_cross])
    
    # Determine entry quality
    entry_signal = "AVOID"
    if score == 3:
        if -2 <= dist_20 <= 5:  # Price within 2% below to 5% above 20-day MA
            entry_signal = "BUY - Ideal pullback entry"
        elif dist_20 > 5:
            entry_signal = "WAIT - Extended above MA"
        else:
            entry_signal = "STRONG BUY - At MA support"
    elif score == 2:
        entry_signal = "WAIT - Partial trend"
    
    return {
        'current_price': current_price,
        'SMA_20': latest['SMA_20'],
        'SMA_50': latest['SMA_50'],
        'SMA_200': latest['SMA_200'],
        'above_50': above_50,
        'above_200': above_200,
        'golden_cross': golden_cross,
        'dist_from_20': dist_20,
        'dist_from_50': dist_50,
        'dist_from_200': dist_200,
        'score': score,
        'max_score': 3,
        'signal': entry_signal
    }


# ============================================================================
# INDICATOR 2: MACD
# ============================================================================

def calculate_macd(df, fast=12, slow=26, signal=9):
    """
    Calculate MACD indicator
    
    Args:
        df (DataFrame): Price data
        fast (int): Fast EMA period (default 12)
        slow (int): Slow EMA period (default 26)
        signal (int): Signal line EMA period (default 9)
    
    Returns:
        DataFrame: Data with MACD columns
    """
    df = df.copy()
    
    # Calculate EMAs
    ema_fast = df['Close'].ewm(span=fast, adjust=False).mean()
    ema_slow = df['Close'].ewm(span=slow, adjust=False).mean()
    
    # MACD line
    df['MACD'] = ema_fast - ema_slow
    
    # Signal line
    df['MACD_signal'] = df['MACD'].ewm(span=signal, adjust=False).mean()
    
    # Histogram
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']
    
    return df


def analyze_macd(df):
    """
    Analyze MACD signals
    
    Returns:
        dict: MACD analysis with signals and scores
    """
    if df is None or df.empty:
        return None
    
    latest = df.iloc[-1]
    prev = df.iloc[-2]
    
    # Current values
    macd = latest['MACD']
    signal = latest['MACD_signal']
    hist = latest['MACD_hist']
    
    # Check conditions
    bullish_crossover = macd > signal
    above_zero = macd > 0
    hist_positive = hist > 0
    hist_growing = hist > prev['MACD_hist']
    
    # Check for recent crossover (within last 10 days)
    recent_cross = False
    crossover_days_ago = None
    for i in range(1, min(11, len(df))):
        if df.iloc[-i]['MACD'] > df.iloc[-i]['MACD_signal'] and df.iloc[-i-1]['MACD'] <= df.iloc[-i-1]['MACD_signal']:
            recent_cross = True
            crossover_days_ago = i
            break
    
    # Score (0-3)
    score = sum([bullish_crossover, above_zero, hist_positive])
    
    # Determine signal
    if bullish_crossover and recent_cross and crossover_days_ago <= 5:
        signal_str = f"STRONG BUY - Fresh crossover ({crossover_days_ago} days ago)"
    elif bullish_crossover and hist_growing:
        signal_str = "BUY - Momentum building"
    elif bullish_crossover:
        signal_str = "HOLD - Bullish but watch for weakening"
    else:
        signal_str = "AVOID - Bearish momentum"
    
    return {
        'MACD': macd,
        'MACD_signal': signal,
        'MACD_hist': hist,
        'bullish_crossover': bullish_crossover,
        'above_zero': above_zero,
        'histogram_positive': hist_positive,
        'histogram_growing': hist_growing,
        'recent_crossover': recent_cross,
        'crossover_days_ago': crossover_days_ago,
        'score': score,
        'max_score': 3,
        'signal': signal_str
    }


# ============================================================================
# INDICATOR 3: RSI
# ============================================================================

def calculate_rsi(df, period=14):
    """
    Calculate RSI (Relative Strength Index)
    
    Args:
        df (DataFrame): Price data
        period (int): RSI period (default 14)
    
    Returns:
        DataFrame: Data with RSI column
    """
    df = df.copy()
    
    # Calculate price changes
    delta = df['Close'].diff()
    
    # Separate gains and losses
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    
    # Calculate RS and RSI
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    return df


def analyze_rsi(df):
    """
    Analyze RSI signals
    
    Returns:
        dict: RSI analysis with signals and scores
    """
    if df is None or df.empty:
        return None
    
    latest = df.iloc[-1]
    rsi = latest['RSI']
    
    # Check conditions
    not_overbought = rsi < 70
    above_50 = rsi > 50
    in_sweet_spot = 40 < rsi < 70
    
    # Score (0-3)
    score = sum([not_overbought, above_50, in_sweet_spot])
    
    # Determine signal
    if 45 <= rsi <= 60:
        signal = "STRONG BUY - Ideal zone"
    elif 40 <= rsi <= 70:
        signal = "BUY - Acceptable zone"
    elif rsi > 70:
        signal = "WAIT - Overbought, pullback likely"
    elif rsi < 40:
        signal = "CAUTION - Oversold, wait for bounce confirmation"
    else:
        signal = "NEUTRAL"
    
    # Classification
    if rsi > 70:
        zone = "Overbought"
    elif rsi < 30:
        zone = "Oversold"
    elif rsi > 50:
        zone = "Bullish"
    else:
        zone = "Bearish"
    
    return {
        'RSI': rsi,
        'not_overbought': not_overbought,
        'above_50': above_50,
        'in_sweet_spot': in_sweet_spot,
        'zone': zone,
        'score': score,
        'max_score': 3,
        'signal': signal
    }


# ============================================================================
# INDICATOR 4: VWMA (Volume Weighted Moving Average)
# ============================================================================

def calculate_vwma(df, period=20):
    """
    Calculate Volume Weighted Moving Average
    
    Args:
        df (DataFrame): Price data with Close and Volume
        period (int): VWMA period (default 20)
    
    Returns:
        DataFrame: Data with VWMA column
    """
    df = df.copy()
    
    # VWMA = Sum(Price * Volume) / Sum(Volume)
    df['VWMA'] = (df['Close'] * df['Volume']).rolling(window=period).sum() / \
                  df['Volume'].rolling(window=period).sum()
    
    # Also calculate average volume
    df['Avg_Volume'] = df['Volume'].rolling(window=period).mean()
    
    return df


def analyze_vwma(df):
    """
    Analyze VWMA and volume patterns
    
    Returns:
        dict: VWMA analysis with signals and scores
    """
    if df is None or df.empty:
        return None
    
    latest = df.iloc[-1]
    current_price = latest['Close']
    vwma = latest['VWMA']
    
    # Check conditions
    above_vwma = current_price > vwma
    vwma_rising = df['VWMA'].iloc[-5:].is_monotonic_increasing if len(df) >= 5 else False
    
    # Volume analysis (last 10 days)
    recent_data = df.tail(10).copy()
    recent_data['Price_Change'] = recent_data['Close'].pct_change()
    
    # Volume on up days vs down days
    up_days = recent_data[recent_data['Price_Change'] > 0]
    down_days = recent_data[recent_data['Price_Change'] < 0]
    
    avg_vol_up = up_days['Volume'].mean() if len(up_days) > 0 else 0
    avg_vol_down = down_days['Volume'].mean() if len(down_days) > 0 else 0
    
    volume_pattern_bullish = avg_vol_up > avg_vol_down
    
    # Current volume vs average
    current_vol = latest['Volume']
    avg_vol = latest['Avg_Volume']
    vol_ratio = current_vol / avg_vol if avg_vol > 0 else 0
    
    # Score (0-3)
    score = sum([above_vwma, vwma_rising, volume_pattern_bullish])
    
    # Determine signal
    if above_vwma and vwma_rising and volume_pattern_bullish:
        signal = "STRONG BUY - Institutional accumulation"
    elif above_vwma and volume_pattern_bullish:
        signal = "BUY - Good volume support"
    elif above_vwma:
        signal = "HOLD - Above VWMA but watch volume"
    else:
        signal = "AVOID - Below VWMA, weak support"
    
    return {
        'current_price': current_price,
        'VWMA': vwma,
        'above_vwma': above_vwma,
        'vwma_rising': vwma_rising,
        'volume_pattern_bullish': volume_pattern_bullish,
        'current_volume': current_vol,
        'avg_volume': avg_vol,
        'volume_ratio': vol_ratio,
        'avg_volume_up_days': avg_vol_up,
        'avg_volume_down_days': avg_vol_down,
        'score': score,
        'max_score': 3,
        'signal': signal
    }


# ============================================================================
# INDICATOR 5: SUPPORT & RESISTANCE
# ============================================================================

def calculate_support_resistance(df, lookback=60):
    """
    Identify support and resistance levels
    
    Args:
        df (DataFrame): Price data
        lookback (int): Days to look back for levels
    
    Returns:
        DataFrame: Data with support/resistance columns
    """
    df = df.copy()
    
    # Rolling highs and lows
    df['Resistance_20'] = df['High'].rolling(window=20).max()
    df['Support_20'] = df['Low'].rolling(window=20).min()
    
    df['Resistance_60'] = df['High'].rolling(window=60).max()
    df['Support_60'] = df['Low'].rolling(window=60).min()
    
    return df


def find_pivot_points(df, window=5):
    """
    Find pivot highs and lows (more sophisticated support/resistance)
    
    Args:
        df (DataFrame): Price data
        window (int): Window for pivot detection
    
    Returns:
        list: Resistance levels, list: Support levels
    """
    if df is None or len(df) < window * 2:
        return [], []
    
    resistance_levels = []
    support_levels = []
    
    # Find pivot highs (resistance)
    for i in range(window, len(df) - window):
        if df['High'].iloc[i] == df['High'].iloc[i-window:i+window+1].max():
            resistance_levels.append(df['High'].iloc[i])
    
    # Find pivot lows (support)
    for i in range(window, len(df) - window):
        if df['Low'].iloc[i] == df['Low'].iloc[i-window:i+window+1].min():
            support_levels.append(df['Low'].iloc[i])
    
    # Get unique levels (cluster similar levels within 1%)
    def cluster_levels(levels):
        if not levels:
            return []
        levels = sorted(levels)
        clustered = [levels[0]]
        for level in levels[1:]:
            if level / clustered[-1] > 1.01:  # More than 1% different
                clustered.append(level)
        return clustered[-5:]  # Return last 5 most recent levels
    
    return cluster_levels(resistance_levels), cluster_levels(support_levels)


def analyze_support_resistance(df):
    """
    Analyze current price relative to support/resistance
    
    Returns:
        dict: Support/resistance analysis
    """
    if df is None or df.empty:
        return None
    
    latest = df.iloc[-1]
    current_price = latest['Close']
    
    # Get recent support/resistance
    resistance_levels, support_levels = find_pivot_points(df, window=5)
    
    # Find nearest levels
    resistance_above = [r for r in resistance_levels if r > current_price]
    support_below = [s for s in support_levels if s < current_price]
    
    nearest_resistance = min(resistance_above) if resistance_above else latest['Resistance_60']
    nearest_support = max(support_below) if support_below else latest['Support_60']
    
    # Calculate distances
    dist_to_resistance = ((nearest_resistance - current_price) / current_price * 100)
    dist_to_support = ((current_price - nearest_support) / current_price * 100)
    
    # Check if at support
    at_support = dist_to_support < 2  # Within 2% of support
    near_resistance = dist_to_resistance < 2  # Within 2% of resistance
    
    # Determine position quality
    if at_support and dist_to_resistance > 5:
        signal = "STRONG BUY - At support with room to resistance"
    elif at_support:
        signal = "BUY - At support but near resistance"
    elif near_resistance:
        signal = "WAIT - Near resistance, likely pullback"
    elif 3 < dist_to_support < 8 and dist_to_resistance > 5:
        signal = "BUY - Good risk/reward setup"
    else:
        signal = "NEUTRAL - No clear level nearby"
    
    # Score based on position
    score = 0
    if at_support:
        score += 2
    elif dist_to_support < 5:
        score += 1
    if dist_to_resistance > 5:
        score += 1
    
    return {
        'current_price': current_price,
        'nearest_resistance': nearest_resistance,
        'nearest_support': nearest_support,
        'dist_to_resistance_%': dist_to_resistance,
        'dist_to_support_%': dist_to_support,
        'at_support': at_support,
        'near_resistance': near_resistance,
        'resistance_levels': resistance_levels[-3:] if len(resistance_levels) >= 3 else resistance_levels,
        'support_levels': support_levels[-3:] if len(support_levels) >= 3 else support_levels,
        'score': score,
        'max_score': 3,
        'signal': signal
    }


# ============================================================================
# MARKET & SECTOR CONTEXT
# ============================================================================

def analyze_market_trend(period="6mo"):
    """
    Analyze overall market trend using S&P 500 (SPY)
    
    Returns:
        dict: Market analysis
    """
    spy_data = get_stock_data("SPY", period=period)
    if spy_data is None:
        return None
    
    spy_data = calculate_moving_averages(spy_data)
    latest = spy_data.iloc[-1]
    
    above_50 = latest['Close'] > latest['SMA_50']
    above_200 = latest['Close'] > latest['SMA_200']
    golden_cross = latest['SMA_50'] > latest['SMA_200']
    
    # Try to get VIX (volatility)
    try:
        vix = yf.Ticker("^VIX")
        vix_data = vix.history(period="5d")
        vix_level = vix_data['Close'].iloc[-1] if not vix_data.empty else None
        low_vix = vix_level < 20 if vix_level else None
    except:
        vix_level = None
        low_vix = None
    
    # Score
    checks = [above_50, above_200, golden_cross]
    if low_vix is not None:
        checks.append(low_vix)
    score = sum(checks)
    total = len(checks)
    
    # Determine market state
    if score >= total * 0.75:
        state = "BULL"
        recommendation = "Aggressive - Take 5-7 positions"
    elif score >= total * 0.5:
        state = "NEUTRAL"
        recommendation = "Selective - Take 3-5 positions"
    else:
        state = "BEAR"
        recommendation = "Defensive - Maximum 2 positions, mostly cash"
    
    return {
        'SPY_price': latest['Close'],
        'SPY_50MA': latest['SMA_50'],
        'SPY_200MA': latest['SMA_200'],
        'above_50': above_50,
        'above_200': above_200,
        'golden_cross': golden_cross,
        'VIX': vix_level,
        'low_vix': low_vix,
        'score': score,
        'total_checks': total,
        'market_state': state,
        'recommendation': recommendation
    }


def compare_relative_strength(ticker, benchmark="SPY", period="6mo"):
    """
    Compare stock performance vs benchmark
    
    Args:
        ticker (str): Stock ticker
        benchmark (str): Benchmark ticker (default SPY)
        period (str): Comparison period
    
    Returns:
        dict: Relative strength analysis
    """
    stock_data = get_stock_data(ticker, period=period)
    bench_data = get_stock_data(benchmark, period=period)
    
    if stock_data is None or bench_data is None or stock_data.empty or bench_data.empty:
        return None
    
    # Calculate returns
    stock_return = ((stock_data['Close'].iloc[-1] / stock_data['Close'].iloc[0]) - 1) * 100
    bench_return = ((bench_data['Close'].iloc[-1] / bench_data['Close'].iloc[0]) - 1) * 100
    
    outperformance = stock_return - bench_return
    
    # Determine signal
    if outperformance > 10:
        signal = "STRONG - Significant outperformance"
    elif outperformance > 0:
        signal = "GOOD - Outperforming market"
    elif outperformance > -10:
        signal = "WEAK - Slight underperformance"
    else:
        signal = "AVOID - Significant underperformance"
    
    return {
        'stock_return_%': stock_return,
        'benchmark_return_%': bench_return,
        'outperformance_%': outperformance,
        'outperforming': outperformance > 0,
        'signal': signal
    }


# ============================================================================
# COMPLETE TECHNICAL ANALYSIS
# ============================================================================

def complete_technical_analysis(ticker, period="1y"):
    """
    Run complete technical analysis on a stock
    
    Args:
        ticker (str): Stock ticker symbol
        period (str): Data period for analysis
    
    Returns:
        dict: Complete technical analysis results
    """
    print(f"\n{'='*70}")
    print(f"TECHNICAL ANALYSIS: {ticker}")
    print(f"{'='*70}\n")
    
    # Fetch data
    df = get_stock_data(ticker, period=period)
    if df is None:
        return None
    
    # Calculate all indicators
    df = calculate_moving_averages(df)
    df = calculate_macd(df)
    df = calculate_rsi(df)
    df = calculate_vwma(df)
    df = calculate_support_resistance(df)
    
    # Analyze each indicator
    ma_analysis = analyze_moving_averages(df)
    macd_analysis = analyze_macd(df)
    rsi_analysis = analyze_rsi(df)
    vwma_analysis = analyze_vwma(df)
    sr_analysis = analyze_support_resistance(df)
    
    # Market context
    market_analysis = analyze_market_trend()
    rel_strength = compare_relative_strength(ticker)
    
    # Calculate total score
    total_score = (
        ma_analysis['score'] +
        macd_analysis['score'] +
        rsi_analysis['score'] +
        vwma_analysis['score'] +
        sr_analysis['score']
    )
    max_score = 15
    
    # Determine overall signal
    score_pct = (total_score / max_score) * 100
    
    if score_pct >= 67:
        overall_signal = "🟢 STRONG BUY"
        action = "Enter position"
    elif score_pct >= 50:
        overall_signal = "🟡 WAIT"
        action = "Need more confirmation"
    else:
        overall_signal = "🔴 AVOID"
        action = "Poor setup"
    
    # Compile results
    results = {
        'ticker': ticker,
        'analysis_date': datetime.now().strftime('%Y-%m-%d'),
        'current_price': ma_analysis['current_price'],
        'moving_averages': ma_analysis,
        'macd': macd_analysis,
        'rsi': rsi_analysis,
        'vwma': vwma_analysis,
        'support_resistance': sr_analysis,
        'market_context': market_analysis,
        'relative_strength': rel_strength,
        'total_score': total_score,
        'max_score': max_score,
        'score_percentage': score_pct,
        'overall_signal': overall_signal,
        'action': action,
        'data': df  # Include full dataframe for further analysis
    }
    
    return results


def print_technical_report(results):
    """
    Print formatted technical analysis report
    
    Args:
        results (dict): Results from complete_technical_analysis()
    """
    if results is None:
        print("No results to display")
        return
    
    print(f"\n{'='*70}")
    print(f"TECHNICAL ANALYSIS REPORT: {results['ticker']}")
    print(f"Date: {results['analysis_date']}")
    print(f"Current Price: ${results['current_price']:.2f}")
    print(f"{'='*70}\n")
    
    # Market Context
    if results['market_context']:
        mc = results['market_context']
        print("MARKET CONTEXT (S&P 500):")
        print(f"  State: {mc['market_state']}")
        print(f"  SPY: ${mc['SPY_price']:.2f}")
        print(f"  Above 50-day MA: {'✓' if mc['above_50'] else '✗'}")
        print(f"  Above 200-day MA: {'✓' if mc['above_200'] else '✗'}")
        print(f"  Golden Cross: {'✓' if mc['golden_cross'] else '✗'}")
        if mc['VIX']:
            print(f"  VIX: {mc['VIX']:.1f}")
        print(f"  Recommendation: {mc['recommendation']}\n")
    
    # Relative Strength
    if results['relative_strength']:
        rs = results['relative_strength']
        print("RELATIVE STRENGTH:")
        print(f"  Stock Return (6mo): {rs['stock_return_%']:.1f}%")
        print(f"  S&P 500 Return: {rs['benchmark_return_%']:.1f}%")
        print(f"  Outperformance: {rs['outperformance_%']:.1f}%")
        print(f"  Signal: {rs['signal']}\n")
    
    # Moving Averages
    ma = results['moving_averages']
    print("1. MOVING AVERAGES:")
    print(f"  20-day MA: ${ma['SMA_20']:.2f} ({ma['dist_from_20']:.1f}% from price)")
    print(f"  50-day MA: ${ma['SMA_50']:.2f} ({ma['dist_from_50']:.1f}% from price)")
    print(f"  200-day MA: ${ma['SMA_200']:.2f} ({ma['dist_from_200']:.1f}% from price)")
    print(f"  Above 50-day: {'✓' if ma['above_50'] else '✗'}")
    print(f"  Above 200-day: {'✓' if ma['above_200'] else '✗'}")
    print(f"  Golden Cross: {'✓' if ma['golden_cross'] else '✗'}")
    print(f"  Score: {ma['score']}/{ma['max_score']}")
    print(f"  Signal: {ma['signal']}\n")
    
    # MACD
    macd = results['macd']
    print("2. MACD:")
    print(f"  MACD: {macd['MACD']:.2f}")
    print(f"  Signal: {macd['MACD_signal']:.2f}")
    print(f"  Histogram: {macd['MACD_hist']:.2f}")
    print(f"  Bullish Crossover: {'✓' if macd['bullish_crossover'] else '✗'}")
    print(f"  Above Zero: {'✓' if macd['above_zero'] else '✗'}")
    if macd['recent_crossover']:
        print(f"  Recent Crossover: {macd['crossover_days_ago']} days ago")
    print(f"  Score: {macd['score']}/{macd['max_score']}")
    print(f"  Signal: {macd['signal']}\n")
    
    # RSI
    rsi = results['rsi']
    print("3. RSI:")
    print(f"  RSI: {rsi['RSI']:.1f}")
    print(f"  Zone: {rsi['zone']}")
    print(f"  Not Overbought (<70): {'✓' if rsi['not_overbought'] else '✗'}")
    print(f"  Above 50: {'✓' if rsi['above_50'] else '✗'}")
    print(f"  In Sweet Spot (40-70): {'✓' if rsi['in_sweet_spot'] else '✗'}")
    print(f"  Score: {rsi['score']}/{rsi['max_score']}")
    print(f"  Signal: {rsi['signal']}\n")
    
    # VWMA
    vwma = results['vwma']
    print("4. VWMA & VOLUME:")
    print(f"  VWMA: ${vwma['VWMA']:.2f}")
    print(f"  Above VWMA: {'✓' if vwma['above_vwma'] else '✗'}")
    print(f"  VWMA Rising: {'✓' if vwma['vwma_rising'] else '✗'}")
    print(f"  Volume Pattern Bullish: {'✓' if vwma['volume_pattern_bullish'] else '✗'}")
    print(f"  Current Volume: {vwma['current_volume']:,.0f}")
    print(f"  Avg Volume: {vwma['avg_volume']:,.0f}")
    print(f"  Volume Ratio: {vwma['volume_ratio']:.2f}x")
    print(f"  Score: {vwma['score']}/{vwma['max_score']}")
    print(f"  Signal: {vwma['signal']}\n")
    
    # Support & Resistance
    sr = results['support_resistance']
    print("5. SUPPORT & RESISTANCE:")
    print(f"  Current Price: ${sr['current_price']:.2f}")
    print(f"  Nearest Support: ${sr['nearest_support']:.2f} (-{sr['dist_to_support_%']:.1f}%)")
    print(f"  Nearest Resistance: ${sr['nearest_resistance']:.2f} (+{sr['dist_to_resistance_%']:.1f}%)")
    print(f"  At Support: {'✓' if sr['at_support'] else '✗'}")
    print(f"  Near Resistance: {'✓' if sr['near_resistance'] else '✗'}")
    print(f"  Score: {sr['score']}/{sr['max_score']}")
    print(f"  Signal: {sr['signal']}\n")
    
    # Overall Score
    print(f"{'='*70}")
    print(f"OVERALL TECHNICAL SCORE: {results['total_score']}/{results['max_score']} ({results['score_percentage']:.0f}%)")
    print(f"SIGNAL: {results['overall_signal']}")
    print(f"ACTION: {results['action']}")
    print(f"{'='*70}\n")
    
    # Entry/Exit Recommendations
    if results['score_percentage'] >= 67:
        print("ENTRY RECOMMENDATION:")
        print(f"  Entry Price: ${results['current_price']:.2f}")
        print(f"  Stop Loss: ${sr['nearest_support'] * 0.97:.2f} (below support)")
        print(f"  Target 1 (+15%): ${results['current_price'] * 1.15:.2f}")
        print(f"  Target 2 (+30%): ${results['current_price'] * 1.30:.2f}")
        print(f"  Risk/Reward: {((results['current_price'] * 1.15 - results['current_price']) / (results['current_price'] - sr['nearest_support'] * 0.97)):.2f}:1\n")


# ============================================================================
# WEEKLY TIMEFRAME ANALYSIS (for longer-term confirmation)
# ============================================================================

def get_weekly_data(ticker, period="2y"):
    """
    Get weekly price data for longer-term analysis
    
    Args:
        ticker (str): Stock ticker
        period (str): Data period (need longer for weekly)
    
    Returns:
        DataFrame: Weekly OHLCV data
    """
    try:
        stock = yf.Ticker(ticker)
        df = stock.history(period=period, interval="1wk")
        
        if df.empty:
            print(f"No weekly data found for {ticker}")
            return None
            
        return df
    except Exception as e:
        print(f"Error fetching weekly data for {ticker}: {e}")
        return None


def analyze_weekly_trend(ticker):
    """
    Analyze weekly timeframe for longer-term trend confirmation
    
    Args:
        ticker (str): Stock ticker
    
    Returns:
        dict: Weekly trend analysis
    """
    df = get_weekly_data(ticker)
    if df is None or df.empty:
        return None
    
    # Calculate weekly indicators
    df = calculate_moving_averages(df)
    df = calculate_macd(df)
    df = calculate_rsi(df)
    
    latest = df.iloc[-1]
    
    # Weekly MA alignment (10/20/40 week = ~50/100/200 day)
    above_10w = latest['Close'] > df['Close'].rolling(10).mean().iloc[-1]
    above_20w = latest['Close'] > df['Close'].rolling(20).mean().iloc[-1]
    
    # Weekly MACD
    weekly_macd_bullish = latest['MACD'] > latest['MACD_signal']
    
    # Weekly RSI
    weekly_rsi = latest['RSI']
    weekly_rsi_bullish = weekly_rsi > 50
    
    # Score
    score = sum([above_10w, above_20w, weekly_macd_bullish, weekly_rsi_bullish])
    
    if score >= 3:
        signal = "STRONG - Weekly uptrend confirmed"
    elif score >= 2:
        signal = "ACCEPTABLE - Weekly trend mixed"
    else:
        signal = "WEAK - Weekly trend not supportive"
    
    return {
        'above_10week_ma': above_10w,
        'above_20week_ma': above_20w,
        'weekly_macd_bullish': weekly_macd_bullish,
        'weekly_rsi': weekly_rsi,
        'weekly_rsi_bullish': weekly_rsi_bullish,
        'score': score,
        'max_score': 4,
        'signal': signal
    }


# ============================================================================
# PORTFOLIO POSITION CALCULATOR
# ============================================================================

def calculate_position_size(portfolio_value, risk_percent, entry_price, stop_loss_price):
    """
    Calculate position size based on risk management
    
    Args:
        portfolio_value (float): Total portfolio value
        risk_percent (float): Risk per trade (e.g., 1.5 for 1.5%)
        entry_price (float): Entry price per share
        stop_loss_price (float): Stop loss price per share
    
    Returns:
        dict: Position sizing details
    """
    # Calculate risk amount
    risk_amount = portfolio_value * (risk_percent / 100)
    
    # Calculate risk per share
    risk_per_share = entry_price - stop_loss_price
    
    if risk_per_share <= 0:
        return {
            'error': 'Stop loss must be below entry price',
            'shares': 0,
            'position_value': 0
        }
    
    # Calculate number of shares
    shares = int(risk_amount / risk_per_share)
    
    # Calculate position value
    position_value = shares * entry_price
    
    # Calculate position as % of portfolio
    position_percent = (position_value / portfolio_value) * 100
    
    # Calculate actual risk (accounting for integer shares)
    actual_risk = shares * risk_per_share
    actual_risk_percent = (actual_risk / portfolio_value) * 100
    
    return {
        'shares': shares,
        'position_value': position_value,
        'position_percent': position_percent,
        'risk_amount': actual_risk,
        'risk_percent': actual_risk_percent,
        'risk_per_share': risk_per_share,
        'risk_reward_ratio': None  # Will calculate with targets
    }


def calculate_targets(entry_price, stop_loss_price, target_percents=[15, 30]):
    """
    Calculate profit targets
    
    Args:
        entry_price (float): Entry price
        stop_loss_price (float): Stop loss price
        target_percents (list): Target percentages
    
    Returns:
        dict: Target prices and R:R ratios
    """
    risk = entry_price - stop_loss_price
    
    targets = {}
    for i, pct in enumerate(target_percents, 1):
        target_price = entry_price * (1 + pct/100)
        reward = target_price - entry_price
        rr_ratio = reward / risk if risk > 0 else 0
        
        targets[f'target_{i}'] = {
            'percent': pct,
            'price': target_price,
            'reward_amount': reward,
            'risk_reward_ratio': rr_ratio
        }
    
    return targets


# ============================================================================
# COMPLETE STOCK EVALUATION (Fundamental + Technical)
# ============================================================================

def complete_stock_evaluation(ticker, fundamental_score, portfolio_value=100000, risk_percent=1.5):
    """
    Complete stock evaluation combining fundamental and technical analysis
    
    Args:
        ticker (str): Stock ticker
        fundamental_score (int): Score from your fundamental analysis (0-100)
        portfolio_value (float): Total portfolio value for position sizing
        risk_percent (float): Risk per trade percentage
    
    Returns:
        dict: Complete evaluation results
    """
    print(f"\n{'='*70}")
    print(f"COMPLETE STOCK EVALUATION: {ticker}")
    print(f"{'='*70}\n")
    
    # Check fundamental score first
    print(f"FUNDAMENTAL SCORE: {fundamental_score}/100")
    if fundamental_score < 70:
        print("❌ REJECTED - Fundamental score below 70")
        print("Must pass fundamental screening before technical analysis")
        return None
    print("✓ PASSED - Fundamental screening\n")
    
    # Market context check
    print("Checking market context...")
    market = analyze_market_trend()
    if market:
        print(f"Market State: {market['market_state']}")
        if market['market_state'] == 'BEAR':
            print("⚠ WARNING - Bear market conditions")
            print("Consider reducing position size or waiting\n")
    
    # Technical analysis
    print("Running technical analysis...")
    tech_results = complete_technical_analysis(ticker)
    
    if tech_results is None:
        print("❌ Unable to complete technical analysis")
        return None
    
    # Weekly confirmation
    print("Checking weekly timeframe...")
    weekly = analyze_weekly_trend(ticker)
    if weekly:
        print(f"Weekly Trend: {weekly['signal']}\n")
    
    # Print technical report
    print_technical_report(tech_results)
    
    # Final recommendation
    print(f"\n{'='*70}")
    print("FINAL RECOMMENDATION")
    print(f"{'='*70}")
    
    tech_score_pct = tech_results['score_percentage']
    
    # Determine action
    if fundamental_score >= 70 and tech_score_pct >= 67:
        if weekly and weekly['score'] >= 2:
            recommendation = "🟢 STRONG BUY"
            action_detail = "All systems green - Enter position"
        else:
            recommendation = "🟡 BUY WITH CAUTION"
            action_detail = "Daily signals good, but weekly trend weak"
    elif fundamental_score >= 70 and tech_score_pct >= 50:
        recommendation = "🟡 WAIT"
        action_detail = "Good fundamentals, but wait for better technical setup"
    else:
        recommendation = "🔴 AVOID"
        action_detail = "Insufficient technical confirmation"
    
    print(f"Recommendation: {recommendation}")
    print(f"Action: {action_detail}\n")
    
    # If buy signal, calculate position
    if tech_score_pct >= 67:
        sr = tech_results['support_resistance']
        entry = tech_results['current_price']
        stop = sr['nearest_support'] * 0.97  # 3% below support
        
        position = calculate_position_size(portfolio_value, risk_percent, entry, stop)
        targets = calculate_targets(entry, stop)
        
        print(f"POSITION SIZING (Portfolio: ${portfolio_value:,.0f}, Risk: {risk_percent}%):")
        print(f"  Shares to Buy: {position['shares']}")
        print(f"  Position Value: ${position['position_value']:,.2f}")
        print(f"  Position Size: {position['position_percent']:.1f}% of portfolio")
        print(f"  Risk Amount: ${position['risk_amount']:,.2f}")
        print(f"  Actual Risk: {position['risk_percent']:.2f}%\n")
        
        print(f"ENTRY/EXIT PLAN:")
        print(f"  Entry Price: ${entry:.2f}")
        print(f"  Stop Loss: ${stop:.2f} (-{((entry-stop)/entry*100):.1f}%)")
        
        for target_name, target_data in targets.items():
            print(f"  Target {target_name[-1]} (+{target_data['percent']}%): ${target_data['price']:.2f} [R:R = {target_data['risk_reward_ratio']:.2f}:1]")
        
        print(f"\nEXIT STRATEGY:")
        print(f"  Sell 30% at Target 1 (+15%)")
        print(f"  Sell 40% at Target 2 (+30%)")
        print(f"  Trail remaining 30% with 20-day MA")
        print(f"  Hard stop: ${stop:.2f}")
    
    print(f"{'='*70}\n")
    
    # Compile complete results
    results = {
        'ticker': ticker,
        'fundamental_score': fundamental_score,
        'technical_results': tech_results,
        'weekly_trend': weekly,
        'market_context': market,
        'recommendation': recommendation,
        'action': action_detail
    }
    
    if tech_score_pct >= 67:
        results['position_sizing'] = position
        results['targets'] = targets
        results['entry_price'] = entry
        results['stop_loss'] = stop
    
    return results


# ============================================================================
# BATCH ANALYSIS FOR MULTIPLE STOCKS
# ============================================================================

def batch_analyze_stocks(stock_list, fundamental_scores):
    """
    Analyze multiple stocks at once
    
    Args:
        stock_list (list): List of ticker symbols
        fundamental_scores (dict): Dictionary of {ticker: score}
    
    Returns:
        list: Results for all stocks sorted by technical score
    """
    all_results = []
    
    print(f"\n{'='*70}")
    print(f"BATCH ANALYSIS: {len(stock_list)} STOCKS")
    print(f"{'='*70}\n")
    
    for ticker in stock_list:
        fund_score = fundamental_scores.get(ticker, 0)
        
        if fund_score < 70:
            print(f"{ticker}: SKIPPED (Fundamental score: {fund_score})\n")
            continue
        
        try:
            results = complete_stock_evaluation(ticker, fund_score)
            if results:
                all_results.append(results)
        except Exception as e:
            print(f"Error analyzing {ticker}: {e}\n")
            continue
    
    # Sort by technical score
    all_results.sort(key=lambda x: x['technical_results']['score_percentage'], reverse=True)
    
    # Print summary
    print(f"\n{'='*70}")
    print(f"BATCH ANALYSIS SUMMARY")
    print(f"{'='*70}\n")
    print(f"{'Ticker':<10} {'Fund Score':<12} {'Tech Score':<12} {'Signal':<20}")
    print(f"{'-'*70}")
    
    for result in all_results:
        ticker = result['ticker']
        fund = result['fundamental_score']
        tech = result['technical_results']['score_percentage']
        signal = result['recommendation']
        
        print(f"{ticker:<10} {fund:<12} {tech:.0f}%{' ':<9} {signal:<20}")
    
    print(f"\n")
    
    return all_results


# ============================================================================
# USAGE EXAMPLES
# ============================================================================

if __name__ == "__main__":
    
    # Example 1: Single stock analysis
    print("EXAMPLE 1: Single Stock Analysis")
    print("="*70)
    
    ticker = "AAPL"
    fundamental_score = 82  # Your fundamental score
    
    results = complete_stock_evaluation(ticker, fundamental_score, portfolio_value=100000, risk_percent=1.5)
    
    
    # Example 2: Check market conditions first
    print("\n\nEXAMPLE 2: Check Market Conditions")
    print("="*70)
    
    market = analyze_market_trend()
    if market:
        print(f"Market State: {market['market_state']}")
        print(f"Recommendation: {market['recommendation']}")
    
    
    # Example 3: Batch analysis
    print("\n\nEXAMPLE 3: Batch Analysis")
    print("="*70)
    
    stock_list = ["AAPL", "MSFT", "GOOGL", "NVDA", "META"]
    
    # Your fundamental scores from your analysis
    fundamental_scores = {
        "AAPL": 82,
        "MSFT": 88,
        "GOOGL": 75,
        "NVDA": 79,
        "META": 72
    }
    
    batch_results = batch_analyze_stocks(stock_list, fundamental_scores)
    
    
    # Example 4: Get just the data with indicators (for custom analysis)
    print("\n\nEXAMPLE 4: Get Data with Indicators")
    print("="*70)
    
    df = get_stock_data("AAPL", period="1y")
    df = calculate_moving_averages(df)
    df = calculate_macd(df)
    df = calculate_rsi(df)
    df = calculate_vwma(df)
    df = calculate_support_resistance(df)
    
    # Now you have a dataframe with all indicators
    print(f"\nData shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print(f"\nLast 5 rows:")
    print(df[['Close', 'SMA_50', 'SMA_200', 'MACD', 'RSI', 'VWMA']].tail())
    
    
    # Example 5: Weekly timeframe check
    print("\n\nEXAMPLE 5: Weekly Timeframe Analysis")
    print("="*70)
    
    weekly_analysis = analyze_weekly_trend("AAPL")
    if weekly_analysis:
        print(f"Weekly Signal: {weekly_analysis['signal']}")
        print(f"Score: {weekly_analysis['score']}/{weekly_analysis['max_score']}")
    
    
    # Example 6: Position sizing only
    print("\n\nEXAMPLE 6: Position Sizing Calculator")
    print("="*70)
    
    portfolio = 100000
    risk_pct = 1.5
    entry = 227.50
    stop = 218.00
    
    position = calculate_position_size(portfolio, risk_pct, entry, stop)
    targets = calculate_targets(entry, stop)
    
    print(f"Portfolio: ${portfolio:,.0f}")
    print(f"Risk: {risk_pct}%")
    print(f"Entry: ${entry:.2f}")
    print(f"Stop: ${stop:.2f}")
    print(f"\nShares: {position['shares']}")
    print(f"Position Value: ${position['position_value']:,.2f}")
    print(f"Position %: {position['position_percent']:.1f}%")
    print(f"\nTargets:")
    for name, data in targets.items():
        print(f"  {name}: ${data['price']:.2f} (R:R = {data['risk_reward_ratio']:.2f}:1)")

EXAMPLE 1: Single Stock Analysis

COMPLETE STOCK EVALUATION: AAPL

FUNDAMENTAL SCORE: 82/100
✓ PASSED - Fundamental screening

Checking market context...
Market State: NEUTRAL
Running technical analysis...

TECHNICAL ANALYSIS: AAPL

Checking weekly timeframe...
Weekly Trend: STRONG - Weekly uptrend confirmed


TECHNICAL ANALYSIS REPORT: AAPL
Date: 2025-10-06
Current Price: $256.69

MARKET CONTEXT (S&P 500):
  State: NEUTRAL
  SPY: $671.61
  Above 50-day MA: ✓
  Above 200-day MA: ✗
  Golden Cross: ✗
  VIX: 16.4
  Recommendation: Selective - Take 3-5 positions

RELATIVE STRENGTH:
  Stock Return (6mo): 41.8%
  S&P 500 Return: 33.9%
  Outperformance: 7.9%
  Signal: GOOD - Outperforming market

1. MOVING AVERAGES:
  20-day MA: $246.70 (4.1% from price)
  50-day MA: $233.53 (9.9% from price)
  200-day MA: $221.83 (15.7% from price)
  Above 50-day: ✓
  Above 200-day: ✓
  Golden Cross: ✓
  Score: 3/3
  Signal: BUY - Ideal pullback entry

2. MACD:
  MACD: 7.22
  Signal: 7.03
  Histogram: 0.19
 

In [5]:
df = get_stock_data("ABNB", period="1y")
df = calculate_moving_averages(df)
df = calculate_macd(df)
df = calculate_rsi(df)
df = calculate_vwma(df)
df = calculate_support_resistance(df)

In [16]:
tech_results

{'ticker': 'ABNB',
 'analysis_date': '2025-10-18',
 'current_price': np.float64(125.91000366210938),
 'moving_averages': {'current_price': np.float64(125.91000366210938),
  'SMA_20': np.float64(121.9590000152588),
  'SMA_50': np.float64(123.7986003112793),
  'SMA_200': np.float64(129.39880050659178),
  'above_50': np.True_,
  'above_200': np.False_,
  'golden_cross': np.False_,
  'dist_from_20': np.float64(3.2396163024920295),
  'dist_from_50': np.float64(1.705514719488879),
  'dist_from_200': np.float64(-2.6961585662493706),
  'score': np.int64(1),
  'max_score': 3,
  'signal': 'AVOID'},
 'macd': {'MACD': np.float64(-0.4344241882233888),
  'MACD_signal': np.float64(-1.1209250797344361),
  'MACD_hist': np.float64(0.6865008915110473),
  'bullish_crossover': np.True_,
  'above_zero': np.False_,
  'histogram_positive': np.True_,
  'histogram_growing': np.True_,
  'recent_crossover': True,
  'crossover_days_ago': 4,
  'score': np.int64(2),
  'max_score': 3,
  'signal': 'STRONG BUY - Fresh 

In [15]:
ticker = "ABNB"
tech_results = complete_technical_analysis(ticker)
    
if tech_results is None:
    print("❌ Unable to complete technical analysis")

# Weekly confirmation
print("Checking weekly timeframe...")
weekly = analyze_weekly_trend(ticker)
if weekly:
    print(f"Weekly Trend: {weekly['signal']}\n")

# Print technical report
print_technical_report(tech_results)


TECHNICAL ANALYSIS: ABNB

Checking weekly timeframe...
Weekly Trend: WEAK - Weekly trend not supportive


TECHNICAL ANALYSIS REPORT: ABNB
Date: 2025-10-18
Current Price: $125.91

MARKET CONTEXT (S&P 500):
  State: BEAR
  SPY: $664.39
  Above 50-day MA: ✓
  Above 200-day MA: ✗
  Golden Cross: ✗
  VIX: 20.8
  Recommendation: Defensive - Maximum 2 positions, mostly cash

RELATIVE STRENGTH:
  Stock Return (6mo): 12.8%
  S&P 500 Return: 30.0%
  Outperformance: -17.2%
  Signal: AVOID - Significant underperformance

1. MOVING AVERAGES:
  20-day MA: $121.96 (3.2% from price)
  50-day MA: $123.80 (1.7% from price)
  200-day MA: $129.40 (-2.7% from price)
  Above 50-day: ✓
  Above 200-day: ✗
  Golden Cross: ✗
  Score: 1/3
  Signal: AVOID

2. MACD:
  MACD: -0.43
  Signal: -1.12
  Histogram: 0.69
  Bullish Crossover: ✓
  Above Zero: ✗
  Recent Crossover: 4 days ago
  Score: 2/3
  Signal: STRONG BUY - Fresh crossover (4 days ago)

3. RSI:
  RSI: 58.4
  Zone: Bullish
  Not Overbought (<70): ✓
  Abo

In [7]:
print(f"\nData shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nLast 5 rows:")
print(df[['Close', 'SMA_50', 'SMA_200', 'MACD', 'RSI', 'VWMA']].tail())


Data shape: (250, 22)
Columns: ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'SMA_20', 'SMA_50', 'SMA_200', 'EMA_20', 'EMA_50', 'MACD', 'MACD_signal', 'MACD_hist', 'RSI', 'VWMA', 'Avg_Volume', 'Resistance_20', 'Support_20', 'Resistance_60', 'Support_60']

Last 5 rows:
                                Close      SMA_50     SMA_200      MACD  \
Date                                                                      
2025-09-30 00:00:00-04:00  121.419998  127.207600  130.176076 -1.221612   
2025-10-01 00:00:00-04:00  122.320000  126.865401  130.093476 -1.211298   
2025-10-02 00:00:00-04:00  121.489998  126.498600  130.013576 -1.255624   
2025-10-03 00:00:00-04:00  120.220001  126.070601  129.959926 -1.377354   
2025-10-06 00:00:00-04:00  120.349998  125.651401  129.900676 -1.446660   

                                 RSI        VWMA  
Date                                              
2025-09-30 00:00:00-04:00  44.825778  123.714408  
2025-10-01 00:00:00-04:00

In [9]:
weekly_analysis = analyze_weekly_trend("ABNB")
if weekly_analysis:
    print(f"Weekly Signal: {weekly_analysis['signal']}")
    print(f"Score: {weekly_analysis['score']}/{weekly_analysis['max_score']}")

Weekly Signal: WEAK - Weekly trend not supportive
Score: 0/4


In [1]:
from technical_analysis import  TechnicalAnalysis

In [2]:
tech_analyzer = TechnicalAnalysis()

In [3]:
res = tech_analyzer.complete_technical_analysis("AAPL")


TECHNICAL ANALYSIS: AAPL



In [4]:
res['data'].columns

KeyError: 'data'

In [18]:
res

{'ticker': 'AAPL',
 'analysis_date': '2025-10-28',
 'current_price': np.float64(269.0),
 'moving_averages': {'current_price': np.float64(269.0),
  'SMA_20': np.float64(256.465998840332),
  'SMA_50': np.float64(245.64839904785157),
  'SMA_200': np.float64(222.45501937866212),
  'above_50': np.True_,
  'above_200': np.True_,
  'golden_cross': np.True_,
  'dist_from_20': np.float64(4.8871979975292055),
  'dist_from_50': np.float64(9.506107527124412),
  'dist_from_200': np.float64(20.9233222749221),
  'score': np.int64(3),
  'max_score': 3,
  'signal': 'BUY - Ideal pullback entry'},
 'macd': {'MACD': np.float64(5.4680491434275496),
  'MACD_signal': np.float64(4.516993841106782),
  'MACD_hist': np.float64(0.9510553023207677),
  'bullish_crossover': np.True_,
  'above_zero': np.True_,
  'histogram_positive': np.True_,
  'histogram_growing': np.True_,
  'recent_crossover': True,
  'crossover_days_ago': 4,
  'score': np.int64(3),
  'max_score': 3,
  'signal': 'STRONG BUY - Fresh crossover (4 d

In [5]:
import pandas as pd
r1 = pd.json_normalize(res)

In [8]:
pd.set_option('display.max_columns', None)

In [9]:
r1

,ticker,analysis_date,current_price,total_score,max_score,score_percentage,overall_signal,action,moving_averages.current_price,moving_averages.SMA_20,moving_averages.SMA_50,moving_averages.SMA_200,moving_averages.above_50,moving_averages.above_200,moving_averages.golden_cross,moving_averages.dist_from_20,moving_averages.dist_from_50,moving_averages.dist_from_200,moving_averages.score,moving_averages.max_score,moving_averages.signal,macd.MACD,macd.MACD_signal,macd.MACD_hist,macd.bullish_crossover,macd.above_zero,macd.histogram_positive,macd.histogram_growing,macd.recent_crossover,macd.crossover_days_ago,macd.score,macd.max_score,macd.signal,rsi.RSI,rsi.not_overbought,rsi.above_50,rsi.in_sweet_spot,rsi.zone,rsi.score,rsi.max_score,rsi.signal,vwma.current_price,vwma.VWMA,vwma.above_vwma,vwma.vwma_rising,vwma.volume_pattern_bullish,vwma.current_volume,vwma.avg_volume,vwma.volume_ratio,vwma.avg_volume_up_days,vwma.avg_volume_down_days,vwma.score,vwma.max_score,vwma.signal,support_resistance.current_price,support_resistance.nearest_resistance,support_resistance.nearest_support,support_resistance.dist_to_resistance_%,support_resistance.dist_to_support_%,support_resistance.at_support,support_resistance.near_resistance,support_resistance.resistance_levels,support_resistance.support_levels,support_resistance.score,support_resistance.max_score,support_resistance.signal,market_context.SPY_price,market_context.SPY_50MA,market_context.SPY_200MA,market_context.above_50,market_context.above_200,market_context.golden_cross,market_context.VIX,market_context.low_vix,market_context.score,market_context.total_checks,market_context.market_state,market_context.recommendation,relative_strength.stock_return_%,relative_strength.benchmark_return_%,relative_strength.outperformance_%,relative_strength.outperforming,relative_strength.signal
0,AAPL,2025-10-28,269.0,12,15,80.0,🟢 STRONG BUY,Enter position,269.0,256.465999,245.648399,222.455019,True,True,True,4.887198,9.506108,20.923322,3,3,BUY - Ideal pullback entry,5.468049,4.516994,0.951055,True,True,True,True,True,4,3,3,STRONG BUY - Fresh crossover (4 days ago),61.176967,True,True,True,Bullish,3,3,BUY - Acceptable zone,269.0,256.702929,True,True,True,41376749.0,44492192.45,0.929978,4.908564e+07,42396150.0,3,3,STRONG BUY - Institutional accumulation,269.0,269.869995,244.0,0.323418,9.29368,False,True,"[246.31561143302469, 249.38956194785018, 259.1...","[218.6039696546368, 223.77999877929688, 244.0]",0,3,"WAIT - Near resistance, likely pullback",687.059998,658.666392,NaN,True,False,False,16.42,True,2,4,NEUTRAL,Selective - Take 3-5 positions,27.673142,24.65752,3.015621,True,GOOD - Outperforming market
